# Calculating tokens for Semantic Search (ELSER and E5)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/document-chunking/tokenization.ipynb)

Elasticsearch offers [semantic search](https://www.elastic.co/what-is/semantic-search) models, most notably [ELSER](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) and [E5](https://www.elastic.co/search-labs/blog/articles/multilingual-vector-search-e5-embedding-model), to search through documents in a way that takes the text's meaning into account. Part of the semantic search process is breaking up texts into tokens (both for documents and for queries). Tokens are commonly thought of as words, but this is not completely accurate. Different semantic models use different concepts of tokens. Many treat punctuation separately and some break up compound words. For example ELSER (our English language model) uses the [`bert-base-uncased`](https://huggingface.co/bert-base-uncased) tokenizer.

For users of Elasticsearch it is important to know how texts are broken up into tokens because currently only the [first 512 tokens per field](https://www.elastic.co/guide/en/machine-learning/8.12/ml-nlp-limitations.html#ml-nlp-elser-v1-limit-512) are considered. This means that when you index longer texts, all tokens after the 512th are ignored in your semantic search. Hence it is valuable to know the number of tokens for your input texts before choosing the right model and indexing method.

Currently it is not possible to get the token count information via the API, so here we share the code for calculating token counts. This notebook also shows how to break longer text up into chunks of the right size so that no information is lost during indexing. Currently (as of version 8.12) this has to be done by the user. Future versions will remove this necessity and Elasticsearch will automatically create chunks behind the scenes.

# Install packages

As stated above, ELSER uses [BERT](https://huggingface.co/blog/bert-101)'s tokenizer internally. Here we install the `transformers` package that gives us an interface to this tokenizer. (We install the `tabulate` packge to be able to print a nice table for comparison later on.)

In [1]:
!pip install -qU tabulate transformers

Next, we import everything we need. You can ignore a potential warning on models not being available because we only need the tokenizer here.

In [2]:
import json
from urllib.request import urlopen

from tabulate import tabulate
from transformers import AutoTokenizer, BertTokenizer

# Define tokenizers

Now we are ready to initialize the BERT tokenizer that ELSER uses and the E5 tokenizer for the multilingual semantic search. We also define a whitespace tokenizer in order to compare the naive version on creating tokens to the two tokenizers.

In [3]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
e5_tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base")


def whitespace_tokenize(text):
    return text.split()

# Load example data

Download the movies example data that is also used in the other search examples.

In [4]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)
movies = json.load(response)

# Compare token counts

Compare the token counts of the different tokenization methods for the descriptions of the movies.

In [5]:
def count_tokens(text):
    whitespace_tokens = len(whitespace_tokenize(text))
    bert_tokens = len(bert_tokenizer.encode(text))
    e5_tokens = len(e5_tokenizer.encode(text))
    return [whitespace_tokens, bert_tokens, e5_tokens, f"{text[:80]}..."]


counts = [count_tokens(movie["plot"]) for movie in movies]

print(tabulate(sorted(counts), ["whitespace", "BERT", "E5", "text"]))

  whitespace    BERT    E5  text
------------  ------  ----  -----------------------------------------------------------------------------------
          16      21    30  An organized crime dynasty's aging patriarch transfers control of his clandestin...
          19      25    32  Two imprisoned men bond over a number of years, finding solace and eventual rede...
          20      25    34  Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven ...
          20      33    36  An insomniac office worker and a devil-may-care soapmaker form an underground fi...
          22      28    27  An undercover cop and a mole in the police attempt to identify each other while ...
          23      26    31  A computer hacker learns from mysterious rebels about the true nature of his rea...
          26      36    42  A thief who steals corporate secrets through the use of dream-sharing technology...
          27      36    42  The lives of two mob hitmen, a boxer, a gan

Notice that both the BERT the E5 tokenizers yields more tokens in every example, in some cases even twice as many. Why is that? Let's look at an example:

In [6]:
example_movie = movies[0]["plot"]
print(example_movie)
print()

movie_tokens = bert_tokenizer.encode(example_movie)
print(str([bert_tokenizer.decode([t]) for t in movie_tokens]))

The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

['[CLS]', 'the', 'lives', 'of', 'two', 'mob', 'hit', '##men', ',', 'a', 'boxer', ',', 'a', 'gangster', 'and', 'his', 'wife', ',', 'and', 'a', 'pair', 'of', 'diner', 'bandits', 'inter', '##t', '##wine', 'in', 'four', 'tales', 'of', 'violence', 'and', 'redemption', '.', '[SEP]']


We can observe:
- There are special tokens `[CLS]` and `[SEP]` to model the the beginning and end of the text. These two extra tokens will become relevant below.
- Punctuations are their own tokens.
- Compound words are split into two tokens, for example `hitmen` becomes `hit` and `##men`.

Given this behavior, it is easy to see how longer texts yield lots of tokens and can quickly get beyond the 512 tokens limitation mentioned above.

# Handling long texts

We saw how to count the number of tokens using the tokenizers from different models. ELSER uses the BERT tokenizer, so when using `.elser_model_2` it internally splits the text with this method.

Currently there is a limitation that [only the first 512 tokens are considered](https://www.elastic.co/guide/en/machine-learning/8.12/ml-nlp-limitations.html#ml-nlp-elser-v1-limit-512). To work around this, we can first split the input text into chunks of 512 tokens and feed the chunks to Elasticsearch separately. Actually, we need to use a limit of 510 to leave space for the two special tokens (`[CLS]` and `[SEP]`) that we saw.

Furthermore, in practice we often see improved performance when using overlapping chunks. With ELSER, we recommend 50% token overlap.

In [7]:
SEMANTIC_SEARCH_TOKEN_LIMIT = 510  # 512 minus space for the 2 special tokens
ELSER_TOKEN_OVERLAP = 0.5  # 50% token overlap between chunks is recommended for ELSER


def chunk(
    text, chunk_size=SEMANTIC_SEARCH_TOKEN_LIMIT, overlap_ratio=ELSER_TOKEN_OVERLAP
):
    step_size = round(chunk_size * (1 - overlap_ratio))

    tokens = bert_tokenizer.encode(text)
    tokens = tokens[1:-1]  # remove special beginning and end tokens

    result = []
    for i in range(0, len(tokens), step_size):
        end = i + chunk_size
        chunk = tokens[i:end]
        result.append(bert_tokenizer.decode(chunk))
        if end >= len(tokens):
            break
    return result

Here we load a longer example text:

In [8]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/datasets/book_summaries_1000_chunked.json"
response = urlopen(url)
book_summaries = json.load(response)

long_text = book_summaries[0]["synopsis"]

Next we tokenize the long text, exclude the special tokens at beginning and end, create chunks of size 510 tokens and map the tokens back to texts.

In [9]:
chunk(long_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (1242 > 512). Running this sequence through the model will result in indexing errors


['old major, the old boar on the manor farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song,\'beasts of england \'. when major dies, two young pigs, snowball and napoleon, assume command and turn his dream into a philosophy. the animals revolt and drive the drunken and irresponsible mr jones from the farm, renaming it " animal farm ". they adopt seven commandments of animal - ism, the most important of which is, " all animals are equal ". snowball attempts to teach the animals reading and writing ; food is plentiful, and the farm runs smoothly. the pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. napoleon takes the pups from the farm dogs and trains them privately. napoleon and snowball struggle for leadership. when snowball announces his plans to build a windmill, napoleon has his dogs chase snowball away and declares himself le

---
And there we go. Now these chunks can be indexed together on the same document in a [nested field](https://www.elastic.co/guide/en/elasticsearch/reference/current/nested.html) and we can be sure the semantic search model considers our whole text.